In [44]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import random

<h2> 유하 사이트에서 유튜버 정보 크롤링 </h2>

In [1]:
#유하 사이트에서 influencer 정보 긁어오기
#빈 데이터 프레임 생성
df = pd.DataFrame(columns=['category','channel','followers','views','cost'])

In [14]:
#selenium
driver = webdriver.Chrome('chromedriver')
delay =3
driver.implicitly_wait(delay)
driver.get('https://www.youha.info/login/')

userId ="아이디"
userpwd = "패스워드"
driver.find_element_by_name('username').send_keys(userId)
driver.find_element_by_name('password').send_keys(userpwd)
driver.find_element_by_xpath('/html/body/div/form/div[1]/input[2]').click()

driver.get('https://www.youha.info/index/')
html=driver.page_source
soup = BeautifulSoup(html,'html.parser')
category = list()
for i in soup.find('div',class_='categoryMenu showMenu').find_all('a', href=re.compile('^/category/')):
    category.append([i.get('href'),i.text.strip()])

category

[['/category/all/', '전체 채널 보기'],
 ['/category/beauty/', '뷰티 · 패션'],
 ['/category/food/', '먹방'],
 ['/category/bj/', 'BJ · 엔터테이너'],
 ['/category/daily/', 'Vlog · 일상'],
 ['/category/game/', '게임'],
 ['/category/it/', 'IT · 전자기기'],
 ['/category/music/', '음악 · 춤'],
 ['/category/cooking/', '요리 · 베이킹'],
 ['/category/travel/', '여행 · 아웃도어'],
 ['/category/animal/', '동물 · 펫'],
 ['/category/sports/', '스포츠 · 건강'],
 ['/category/movie/', '영화 · 애니'],
 ['/category/car/', '자동차'],
 ['/category/kids/', '키즈 · 플레이'],
 ['/category/education/', '교육'],
 ['/category/politics/', '정치 · 시사'],
 ['/category/fortune/', '운세 · 타로'],
 ['/category/stock/', '부동산 · 투자'],
 ['/category/painting/', '그림 · 페인팅'],
 ['/category/etc/', '기타']]

In [15]:
#필요없는 카테고리 제거
del category[20]
del category[16:19]
del category[9:11]
del category[0]
category

[['/category/beauty/', '뷰티 · 패션'],
 ['/category/food/', '먹방'],
 ['/category/bj/', 'BJ · 엔터테이너'],
 ['/category/daily/', 'Vlog · 일상'],
 ['/category/game/', '게임'],
 ['/category/it/', 'IT · 전자기기'],
 ['/category/music/', '음악 · 춤'],
 ['/category/cooking/', '요리 · 베이킹'],
 ['/category/sports/', '스포츠 · 건강'],
 ['/category/movie/', '영화 · 애니'],
 ['/category/car/', '자동차'],
 ['/category/kids/', '키즈 · 플레이'],
 ['/category/education/', '교육'],
 ['/category/painting/', '그림 · 페인팅']]

In [38]:
idx =0
for cate in category:
    driver.get('https://www.youha.info{0}'.format(cate[0]))
    cate_name = cate[1]
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    count=1
    #페이지 카운팅 copy selector
    for i in soup.select('body > div > div.fit > div > div > a '):
        count +=1
    driver.implicitly_wait(delay)
    #print("count {0}".format(count))
    if count > 30:
        count = 30
    for i in range(2,count-1):
        driver.get('https://www.youha.info{0}?page={1}'.format(cate[0],i))
        driver.implicitly_wait(delay)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        for i in [2,12]:
            channelname =  soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(4) > div > a:nth-child(1) '.format(i))
            followers = soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(5)'.format(i))
            views = soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(7) '.format(i))
            cost =  soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(9) '.format(i))
            #print(channelname[0].text.strip())
            #print(followers[0].text.strip())
            #print(views[0].text.strip()+' ' +comments[0].text.strip() +' ' + cost[0].text.strip())
            #데이터 프레임에 row 행 추가
            df.loc[idx] = [cate_name,channelname[0].text.strip(), followers[0].text.strip(), views[0].text.strip(),cost[0].text.strip() ]
            idx +=1
        driver.implicitly_wait(delay+2)
    
df


,category,channel,followers,views,cost
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000"
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000"
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000"
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000"
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000"
...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200"
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200"
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400"
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000"


In [40]:
df.to_csv('data/influencer.csv',index=False)

In [37]:
df = pd.read_csv('data/influencer.csv')
df

,category,channel,followers,views,cost,category_id
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000
...,...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000


In [7]:
import numpy as np

category= pd.read_csv('data/category.csv')
category
df['category_id']  =np.nan

,category_id,category,parent_id,code
0,100000,외식업,0,0
1,101000,한식음식점,100000,0
2,101001,한식/백반/한정식,101000,Q01A01
3,101002,갈비/삼겹살,101000,Q01A02
4,101003,곱창/양구이전문,101000,Q01A03
...,...,...,...,...
274,701016,교복판매,701000,D05A16
275,702000,패션잡화,700000,0
276,702001,섬유/직물/주단,702000,D05A18
277,702002,특수복판매,702000,D05A19


In [24]:

for idx in df.index:
    if(df.iloc[idx]['category']=='뷰티 · 패션'):
        df['category_id'][idx]= '500000'
    elif(df.iloc[idx]['category']=='먹방'):
        df['category_id'][idx]= '100000'
    elif(df.iloc[idx]['category']=='BJ · 엔터테이너'):
        df['category_id'][idx]= '800000'
    elif(df.iloc[idx]['category']=='Vlog · 일상'):
        df['category_id'][idx]= '800000'
    elif(df.iloc[idx]['category']=='게임'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='IT · 전자기기'):
        df['category_id'][idx]= '600000'
    elif(df.iloc[idx]['category']=='음악 · 춤'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='요리 · 베이킹'):
        df['category_id'][idx]= '100000'
    elif(df.iloc[idx]['category']=='스포츠 · 건강'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='영화 · 애니'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='자동차'):
        df['category_id'][idx]= '400000'
    elif(df.iloc[idx]['category']=='키즈 · 플레이'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='교육'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='그림 · 페인팅'):
        df['category_id'][idx]= '200000'

//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [39]:
#float to int
print(df.dtypes)
df['category_id']= df['category_id'].astype('int')
df

category       object
channel        object
followers      object
views          object
cost           object
category_id     int64
dtype: object


,category,channel,followers,views,cost,category_id
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000
...,...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000


In [40]:
df.to_csv('data/influencer.csv',index=False)

<h2> 인플루언서 분석 데이터 임의로 지정 </h2>

In [66]:
influencers= pd.read_csv('./data/influencer.csv')

<b>인플루언서 타겟팅 대상 랜덤 지정 </b>

In [7]:
influencers = pd.concat([influencers,pd.DataFrame(columns =('target_m','target_w','target_10','target_20','target_30','target_40','target_50','target_60','addr_code'))],sort=False)
for i in influencers.index:
    m = random.randrange(1,100)
    w = 100 -m
    influencers['target_m'][i] = m
    influencers['target_w'][i] = w
    agelist = list()
    temp = 100
    for j in range(1,7):
        age = random.randrange(0,temp)
        agelist.append(age)
        temp = temp - age
    random.shuffle(agelist)
    #print(agelist)
    #print(agelist[0])
    influencers['target_10'][i] = agelist[0]
    influencers['target_20'][i] = agelist[1]
    influencers['target_30'][i] = agelist[2]
    influencers['target_40'][i] = agelist[3]
    influencers['target_50'][i] = agelist[4]
    influencers['target_60'][i] = agelist[5]
influencers

//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/envs/recommender/li

,category,channel,followers,views,cost,category_id,target_m,target_w,target_10,target_20,target_30,target_40,target_50,target_60,addr_code
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000.0,61,39,7,19,61,8,3,1,NaN
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000.0,62,38,0,95,1,2,1,0,NaN
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000.0,33,67,0,44,54,0,1,0,NaN
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000.0,24,76,4,0,53,0,9,33,NaN
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000.0,80,20,5,3,82,0,0,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000.0,88,12,61,16,0,11,11,0,NaN
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000.0,54,46,9,10,10,53,1,14,NaN
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000.0,16,84,19,37,1,10,19,0,NaN
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000.0,22,78,25,21,0,32,11,3,NaN


<b> 인플루언서 활동 지역 임의 지정하기 </b>

In [144]:
stores = pd.read_csv('./data/preprocessing_material/seoulstores.csv')
addr = pd.unique(stores['행정동코드'])
addr.shape

(424,)

In [145]:
size = addr.shape[0]
for i in influencers.index:
    influencers['addr_code'][i] = addr[random.randrange(0,size)]

//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [146]:
#influencers.dtypes
influencers = influencers.astype({'category_id': 'int'})
influencers

,channel_id,category,channel,followers,views,cost,category_id,target_m,target_w,target_10,target_20,target_30,target_40,target_50,target_60,addr_code,addr_city,addr_gu,addr_dong
0,0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000,61.0,39.0,7.0,19.0,61.0,8.0,3.0,1.0,1153055000,서울시,마포구,공덕동
1,1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000,62.0,38.0,0.0,95.0,1.0,2.0,1.0,0.0,1141052000,서울시,종로구,중학동
2,2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000,33.0,67.0,0.0,44.0,54.0,0.0,1.0,0.0,1168054500,서울시,서대문구,홍은동
3,3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000,24.0,76.0,4.0,0.0,53.0,0.0,9.0,33.0,1153076000,서울시,서초구,내곡동
4,4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000,80.0,20.0,5.0,3.0,82.0,0.0,0.0,8.0,1126057000,서울시,종로구,평창동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000,88.0,12.0,61.0,16.0,0.0,11.0,11.0,0.0,1150061500,서울시,중구,중림동
582,582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000,54.0,46.0,9.0,10.0,10.0,53.0,1.0,14.0,1135061200,서울시,종로구,부암동
583,583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000,16.0,84.0,19.0,37.0,1.0,10.0,19.0,0.0,1165059000,서울시,마포구,대흥동
584,584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000,22.0,78.0,25.0,21.0,0.0,32.0,11.0,3.0,1121571000,서울시,종로구,익선동


In [149]:
influencers = influencers.reset_index()
influencers.rename(columns={influencers.columns[0]:"channel_id"}, inplace = True)
influencers

,channel_id,channel_id,channel_id,category,channel,followers,views,cost,category_id,target_m,...,target_10,target_20,target_30,target_40,target_50,target_60,addr_code,addr_city,addr_gu,addr_dong
0,0,0,0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000,61.0,...,7.0,19.0,61.0,8.0,3.0,1.0,1153055000,서울시,마포구,공덕동
1,1,1,1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000,62.0,...,0.0,95.0,1.0,2.0,1.0,0.0,1141052000,서울시,종로구,중학동
2,2,2,2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000,33.0,...,0.0,44.0,54.0,0.0,1.0,0.0,1168054500,서울시,서대문구,홍은동
3,3,3,3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000,24.0,...,4.0,0.0,53.0,0.0,9.0,33.0,1153076000,서울시,서초구,내곡동
4,4,4,4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000,80.0,...,5.0,3.0,82.0,0.0,0.0,8.0,1126057000,서울시,종로구,평창동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,581,581,581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000,88.0,...,61.0,16.0,0.0,11.0,11.0,0.0,1150061500,서울시,중구,중림동
582,582,582,582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000,54.0,...,9.0,10.0,10.0,53.0,1.0,14.0,1135061200,서울시,종로구,부암동
583,583,583,583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000,16.0,...,19.0,37.0,1.0,10.0,19.0,0.0,1165059000,서울시,마포구,대흥동
584,584,584,584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000,22.0,...,25.0,21.0,0.0,32.0,11.0,3.0,1121571000,서울시,종로구,익선동


In [43]:
influencers.to_csv('data/influencer.csv',index=False)

<h3> 시 군 구 임의 입력 </h3>

In [ ]:
influencers = pd.concat([influencers,pd.DataFrame(columns =('addr_city','addr_gu','addr_dong'))],sort=False)

In [150]:
addr= stores.drop_duplicates('행정동코드')
# 시 / 군 / 구 데이터 입력
addr=pd.concat([addr,pd.DataFrame(columns =('addr_city','addr_gu','addr_dong'))],sort=False)

In [151]:
addr= addr.reset_index(drop=True)
addr

,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,addr_city,addr_gu,addr_dong
0,코리아선릉로93길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,11.0,...,135080.0,6149.0,NaN,1,NaN,127.047883,37.505675,NaN,NaN,NaN
1,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11.0,...,153010.0,8545.0,NaN,1,NaN,126.899220,37.471711,NaN,NaN,NaN
2,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11.0,...,135280.0,6280.0,NaN,1,NaN,127.061026,37.493922,NaN,NaN,NaN
3,NaN,F,생활서비스,F09,대중목욕탕/휴게,F09A02,사우나/증기탕/온천,S96121,욕탕업,11.0,...,138846.0,5690.0,NaN,1,NaN,127.104071,37.500249,NaN,NaN,NaN
4,NaN,Q,음식,Q07,패스트푸드,Q07A10,토스트전문,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",11.0,...,143200.0,4992.0,NaN,1,NaN,127.088387,37.549245,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,NaN,R,학문/교육,R10,도서관/독서실,R10A02,고시원,R90212,독서실 운영업,11.0,...,122080.0,3450.0,NaN,4,NaN,126.909912,37.597046,NaN,NaN,NaN
420,NaN,D,소매,D02,선물/팬시/기념품,D02A02,꽃집/꽃배달,G47851,화초 및 산식물 소매업,11.0,...,121120.0,4092.0,NaN,NaN,NaN,126.932918,37.543419,NaN,NaN,NaN
421,NaN,F,생활서비스,F11,개인서비스,F11A01,탐정/경호,NaN,NaN,11.0,...,138210.0,5791.0,NaN,1,NaN,127.136015,37.482087,NaN,NaN,NaN
422,NaN,Q,음식,Q03,일식/수산물,Q03A99,수산물전문음식점-종합,I56111,한식 음식점업,11.0,...,138701.0,5699.0,NaN,1,NaN,127.108696,37.494526,NaN,NaN,NaN


In [152]:
import requests, json
headers = {
        'Authorization': '키값',
    }
for idx in addr.index:
    data = {
        'query' : addr.loc[idx,'지번주소']
    }
    response = requests.get('https://dapi.kakao.com/v2/local/search/address.json', headers=headers, data=data)
    j = json.loads(response.text)
    if j.get('meta').get('pageable_count') ==0:
        continue
    addr['addr_city'][idx] = j.get("documents")[0].get("address").get('region_1depth_name') +'시'
    addr['addr_gu'][idx] = j.get("documents")[0].get("address").get('region_2depth_name')
    addr['addr_dong'][idx] = j.get("documents")[0].get("address").get('region_3depth_h_name')

//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app

In [153]:
addr = addr.astype({'행정동코드':'int'})
addr = addr.set_index('행정동코드')
addr = addr.sort_index()
addr.head()

,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,addr_city,addr_gu,addr_dong
행정동코드,,,,,,,,,,,,,,,,,,,,,
1111051500,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,11.0,...,110030.0,3047.0,NaN,1,NaN,126.970102,37.586243,서울시,종로구,청운효자동
1111053000,경복궁점,Q,음식,Q03,일식/수산물,Q03A08,매운탕전문,I56111,한식 음식점업,11.0,...,110044.0,3041.0,NaN,1,NaN,126.970295,37.576315,서울시,종로구,사직동
1111054000,NaN,Q,음식,Q01,한식,Q01A04,한정식전문,I56111,한식 음식점업,11.0,...,110200.0,3053.0,NaN,1,NaN,126.981613,37.579927,서울시,종로구,삼청동
1111055000,NaN,S,의료,S03,유사의료업,S03B03,침구원,Q86902,유사 의료업,11.0,...,110830.0,3019.0,NaN,1,NaN,126.961868,37.601912,서울시,종로구,부암동
1111056000,NaN,Q,음식,Q08,제과제빵떡케익,Q08A03,떡전문,I56191,제과점업,11.0,...,110847.0,3009.0,NaN,NaN,102,126.967037,37.606444,서울시,종로구,평창동


In [154]:
for idx in influencers.index:
    influencers.loc[idx,['addr_city','addr_gu','addr_dong']] = addr.loc[influencers.loc[idx,'addr_code'],['addr_city','addr_gu','addr_dong']]

In [167]:
influencers = influencers.astype({'channel_id':'int','category_id':'int', 'addr_code':'int'})
influencers

,channel_id,category,channel,followers,views,cost,category_id,target_m,target_w,target_10,target_20,target_30,target_40,target_50,target_60,addr_code,addr_city,addr_gu,addr_dong
0,0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000,61.0,39.0,7.0,19.0,61.0,8.0,3.0,1.0,1153055000,서울시,구로구,구로4동
1,1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000,62.0,38.0,0.0,95.0,1.0,2.0,1.0,0.0,1141052000,서울시,서대문구,천연동
2,2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000,33.0,67.0,0.0,44.0,54.0,0.0,1.0,0.0,1168054500,서울시,강남구,압구정동
3,3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000,24.0,76.0,4.0,0.0,53.0,0.0,9.0,33.0,1153076000,서울시,구로구,개봉3동
4,4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000,80.0,20.0,5.0,3.0,82.0,0.0,0.0,8.0,1126057000,서울시,중랑구,면목7동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000,88.0,12.0,61.0,16.0,0.0,11.0,11.0,0.0,1150061500,서울시,강서구,우장산동
582,582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000,54.0,46.0,9.0,10.0,10.0,53.0,1.0,14.0,1135061200,서울시,노원구,하계2동
583,583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000,16.0,84.0,19.0,37.0,1.0,10.0,19.0,0.0,1165059000,서울시,서초구,방배본동
584,584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000,22.0,78.0,25.0,21.0,0.0,32.0,11.0,3.0,1121571000,서울시,광진구,화양동


In [165]:
#influencers = influencers.drop(influencers.columns[[0,2]], axis='columns')
#influencers.rename(columns={influencers.columns[0]:"channel_id"}, inplace = True)
#influencers

,channel_id,category,channel,followers,views,cost,category_id,target_m,target_w,target_10,target_20,target_30,target_40,target_50,target_60,addr_code,addr_city,addr_gu,addr_dong
0,0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000,61.0,39.0,7.0,19.0,61.0,8.0,3.0,1.0,1153055000,서울시,구로구,구로4동
1,1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000,62.0,38.0,0.0,95.0,1.0,2.0,1.0,0.0,1141052000,서울시,서대문구,천연동
2,2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000,33.0,67.0,0.0,44.0,54.0,0.0,1.0,0.0,1168054500,서울시,강남구,압구정동
3,3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000,24.0,76.0,4.0,0.0,53.0,0.0,9.0,33.0,1153076000,서울시,구로구,개봉3동
4,4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000,80.0,20.0,5.0,3.0,82.0,0.0,0.0,8.0,1126057000,서울시,중랑구,면목7동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000,88.0,12.0,61.0,16.0,0.0,11.0,11.0,0.0,1150061500,서울시,강서구,우장산동
582,582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000,54.0,46.0,9.0,10.0,10.0,53.0,1.0,14.0,1135061200,서울시,노원구,하계2동
583,583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000,16.0,84.0,19.0,37.0,1.0,10.0,19.0,0.0,1165059000,서울시,서초구,방배본동
584,584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000,22.0,78.0,25.0,21.0,0.0,32.0,11.0,3.0,1121571000,서울시,광진구,화양동


In [166]:
influencers.to_csv('./data/influencer.csv',index=False)